# An Analysis of the WMJI Majic 105.7 Top 500 Songs of 1995

**Abstract:** the results of an exploratory analysis of the "Majic Number" list of top 500 songs as found on the April 28, 1998 snapshot of the WMJI radio station website.

## Fetch the Data

In [31]:
from bs4 import BeautifulSoup
import requests
import pickle
from datetime import datetime
import sys

In [ ]:
# Toggle this to False if you need to fetch the
# data for the first time
useCache = True

In [2]:
# The root web page from which to download the data set
url = 'http://web.archive.org/web/19990428111902/http://www.wmji.com/fullsite/top500/top500.html'

In [3]:
response = requests.get(url)

In [13]:
# Parse the result and enumerate links to follow
rootSoup = BeautifulSoup(response.content.decode())

In [14]:
# Collect the URLs to the data set
links = [ link for link in rootSoup.find_all('a') if "songsnumberpt" in link['href'] ]

In [16]:
aLink = links[0]
aLink

<a href="songsnumberpt1.html">1-125</a>

In [24]:
# Collect and parse the pages at each link
listOfSoups = list()

rootPage = url.split('/')
del rootPage[-1]
rootPage = '/'.join(rootPage)+'/'

fetchTime = datetime.utcnow()

for link in links:
    thisLink = rootPage+link['href']
    response = requests.get(thisLink)
    soup = BeautifulSoup(response.content.decode())
    listOfSoups.append(soup)

In [29]:
def writeCache(listOfBeautifulSoups, timestamp, cachePrefix='',
              cacheSuffix='.pickle' ):
    '''writeCache(): ListOfBeautifulSoup datetime.datetime Str1 Str2 -> <File>
    Purpose: to write the LisfOfBeautifulSoup to a cache file in pickle
             format. datetime.datetime is the timestamp (in UTC) at 
             which the data were fetched, and is written into the name 
             of <file>. Optional keywords cachePrefix and cacheSuffix
             set the start and end of the cache file's name.
    '''
    path = cachePrefix + str(datetime.utcnow()).replace(' ','T') + cacheSuffix
    with open(path, 'wb') as rawCache:
        defaultRecLimit = sys.getrecursionlimit()
        sys.setrecursionlimit(10**6)
        pickle.dump(listOfBeautifulSoups,rawCache)
        sys.setrecursionlimit(defaultRecLimit)

In [32]:
writeCache(listOfSoups,fetchTime)

In [33]:
def readCache(path):
    '''readCache(): [Str] --> ListOfSoups
    Purpose: to read in the cached copy of listOfSoups from 
             its pickled cache file. Optional Str argument "path" 
             is alternative path to the cache file.
    '''
    with open(path,'rb') as rawCache:
        listOfRawReports = pickle.load(rawCache)
    
    return listOfSoups